# LSP Client with pygls lib

Communicate with Language Server with LSP, so that many functionalities is available such as get the definition location of target function, get the hover information of target element, get incoming or outgoing calls..


## Environment
* python 3.8 - 3.10
* latest pygls: `pip install pygls==latest`

## References
* Test Cases in [pygls-repository](https://github.com/openlawlibrary/pygls)
* Type definition and API in [Documentation](https://pygls.readthedocs.io/en/latest/)
* Official [LSP documentation](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#initialize)
* LSP Server [List](https://microsoft.github.io/language-server-protocol/implementors/servers/) of different languages.

In [1]:
import pygls.lsp.client as lsp_client
from pygls import uris
from pygls.protocol import LanguageServerProtocol, default_converter
import pygls
import lsprotocol.types as lsp_types

from typing import Dict, List, Type
from concurrent.futures import Future
import asyncio
import os
import logging

logger = logging.getLogger(__name__)

## Create Language Client
[Reference](https://github.com/openlawlibrary/pygls/blob/8e1e8fa3b1ab16fcb804f7d5330ece2e5583206b/tests/client.py#L162)

In [2]:
class LanguageClientProtocol(LanguageServerProtocol):
    """An extended protocol class with extra methods that are useful for testing."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self._notification_futures = {}

    def _handle_notification(self, method_name, params):
        if method_name == lsp_types.CANCEL_REQUEST:
            self._handle_cancel_notification(params.id)
            return

        future = self._notification_futures.pop(method_name, None)
        if future:
            future.set_result(params)

        try:
            handler = self._get_handler(method_name)
            self._execute_notification(handler, params)
        except (KeyError, pygls.exceptions.JsonRpcMethodNotFound):
            logger.warning("Ignoring notification for unknown method '%s'", method_name)
        except Exception:
            logger.exception(
                "Failed to handle notification '%s': %s", method_name, params
            )

    def wait_for_notification(self, method: str, callback=None):
        future: Future = Future()
        if callback:

            def wrapper(future: Future):
                result = future.result()
                callback(result)

            future.add_done_callback(wrapper)

        self._notification_futures[method] = future
        return future

    def wait_for_notification_async(self, method: str):
        future = self.wait_for_notification(method)
        return asyncio.wrap_future(future)

In [3]:
class LanguageClient(lsp_client.BaseLanguageClient):
    """Language client used to drive test cases."""

    def __init__(
        self,
        name: str,
        version: str,
        protocol_cls: Type[LanguageClientProtocol] = LanguageClientProtocol,
        converter_factory=default_converter,
        **kwargs,
    ):
        super().__init__(
            name=name,
            version=version,
            protocol_cls=protocol_cls,
            converter_factory=converter_factory,
            **kwargs,
        )

        self.diagnostics: Dict[str, List[lsp_types.Diagnostic]] = {}
        """Used to hold any recieved diagnostics."""

        self.messages: List[lsp_types.ShowMessageParams] = []
        """Holds any received ``window/showMessage`` requests."""

        self.log_messages: List[lsp_types.LogMessageParams] = []
        """Holds any received ``window/logMessage`` requests."""

    async def wait_for_notification(self, method: str):
        """Block until a notification with the given method is received.

        Parameters
        ----------
        method
           The notification method to wait for, e.g. ``textDocument/publishDiagnostics``
        """
        return await self.protocol.wait_for_notification_async(method)

In [19]:
client = LanguageClient(name="rust_lsp_client", version="v1")

# Register LSP Features

@client.feature(lsp_types.TEXT_DOCUMENT_PUBLISH_DIAGNOSTICS)
def publish_diagnostics(
    client: LanguageClient, params: lsp_types.PublishDiagnosticsParams
):
    client.diagnostics[params.uri] = params.diagnostics


@client.feature(lsp_types.WINDOW_LOG_MESSAGE)
def log_message(client: LanguageClient, params: lsp_types.LogMessageParams):
    client.log_messages.append(params)

    levels = ["ERROR: ", "WARNING: ", "INFO: ", "LOG: "]
    log_level = levels[params.type.value - 1]

    print(log_level, params.message)


@client.feature(lsp_types.WINDOW_SHOW_MESSAGE)
def show_message(client: LanguageClient, params):
    client.messages.append(params)


# cmd to activate target Language Server. `rust-analyzer` is already enough for Rust.
await client.start_io("rust-analyzer")

In [21]:
rust_project_dir = "/mnt/vm_data/pythonProjects/rust_analyzer_client/rust_example/"

init_params = lsp_types.InitializeParams(
    root_path=rust_project_dir,
    capabilities=lsp_types.ClientCapabilities(),  # the LSP api is valid as long as the server support, no need to declare capabilities here
    trace=lsp_types.TraceValues("messages"),
)

init_result = await client.initialize_async(init_params)
init_result

InitializeResult(capabilities=ServerCapabilities(position_encoding='utf-16', text_document_sync=TextDocumentSyncOptions(open_close=True, change=<TextDocumentSyncKind.Incremental: 2>, will_save=None, will_save_wait_until=None, save=SaveOptions(include_text=None)), notebook_document_sync=None, completion_provider=CompletionOptions(trigger_characters=[':', '.', "'", '('], all_commit_characters=None, resolve_provider=None, completion_item=CompletionOptionsCompletionItemType(label_details_support=False), work_done_progress=None), hover_provider=True, signature_help_provider=SignatureHelpOptions(trigger_characters=['(', ',', '<'], retrigger_characters=None, work_done_progress=None), declaration_provider=True, definition_provider=True, type_definition_provider=True, implementation_provider=True, references_provider=True, document_highlight_provider=True, document_symbol_provider=True, code_action_provider=True, code_lens_provider=CodeLensOptions(resolve_provider=True, work_done_progress=None)

In [22]:
client.initialized(init_params)

### Init file uri
The format of file expression is URL, which adds `file://` prefix to identify local files. (So the Language Server should also support analysing file in the Internet via `http`, `ftp`, etc.)

In [23]:
file = "src/main.rs"

file_uri = uris.from_fs_path(os.path.join(rust_project_dir, file))
file_uri

'file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs'

## Text Document Capabilities


### Go to Definition
Navigates to the definition of an identifier.

In [26]:
# definition_provider = init_result.capabilities.definition_provider
symbol_position = lsp_types.Position(line=79, character=9)  # struct Numbered

response = await client.text_document_definition_async(
    lsp_types.DefinitionParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
response

[file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15]

### Go to Decaration
This is the same as `Go to Definition` with the following exceptions: - outline modules will navigate to the mod name; item declaration - trait assoc items will navigate to the assoc item of the trait declaration opposed to the trait impl - fields in patterns will navigate to the field declaration of the struct, union or variant

In [11]:
# declaration_provider = init_result.capabilities.declaration_provider
symbol_position = lsp_types.Position(line=79, character=9)  # struct Numbered

response = await client.text_document_declaration_async(
    lsp_types.DeclarationParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
response

[file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15]

### Go to Type Definition

In [17]:
symbol_position = lsp_types.Position(line=79, character=10)  # struct Numbered

response = await client.text_document_type_definition_async(
    lsp_types.TypeDefinitionParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
response

[file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15]

### Go to Implementation
Navigates to the impl blocks of types.

In [12]:
# impl_provider = init_result.capabilities.implementation_provider
symbol_position = lsp_types.Position(line=79, character=9)  # struct Numbered

response = await client.text_document_implementation_async(
    lsp_types.ImplementationParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
response

[file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:45:22-45:33,
 file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:34:0-34:16]

### Get Hover Information

In [15]:
symbol_position = lsp_types.Position(line=27, character=70)  # struct Numbered

response = await client.text_document_hover_async(
    lsp_types.HoverParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
print(response.contents.value)

core::num::nonzero::NonZeroUsize

pub const unsafe fn new_unchecked(n: usize) -> Self


Creates a non-zero without checking whether the value is non-zero. This results in undefined behaviour if the value is zero.

SafetyThe value must not be zero.


### Get parent component 
Get parent block of current item via `textDocument/selectionRange`.

In [14]:
positions = [
    lsp_types.Position(line=51, character=18),
]
response = await client.text_document_selection_range_async(
    lsp_types.SelectionRangeParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        positions=positions,
    )
)
selection_range = response[0]
while selection_range is not None:
    print(selection_range.range)
    selection_range = selection_range.parent

51:18-51:18
51:18-51:22
51:18-51:31
51:8-51:32
50:67-54:5
46:4-54:5
45:34-73:1
45:0-73:1
0:0-89:0


### Inlay Hints

In [11]:
symbol_position = lsp_types.Position(line=79, character=9)  # struct Numbered
block_range = lsp_types.Range(
    lsp_types.Position(line=50, character=67), lsp_types.Position(line=54, character=5)
)
response = await client.text_document_inlay_hint_async(
    lsp_types.InlayHintParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        range=block_range,
    )
)
response

[InlayHint(position=51:15, label=[InlayHintLabelPart(value=': *const ', tooltip=None, location=None, command=None), InlayHintLabelPart(value='Numbered', tooltip=None, location=file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15, command=None), InlayHintLabelPart(value='<T>', tooltip=None, location=None, command=None)], kind=<InlayHintKind.Type: 1>, text_edits=[TextEdit(range=51:15-51:15, new_text=': *const Numbered<T>')], tooltip=None, padding_left=False, padding_right=False, data=None),
 InlayHint(position=52:13, label=[InlayHintLabelPart(value=': *mut ', tooltip=None, location=None, command=None), InlayHintLabelPart(value='Numbered', tooltip=None, location=file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15, command=None), InlayHintLabelPart(value='<T>', tooltip=None, location=None, command=None)], kind=<InlayHintKind.Type: 1>, text_edits=[TextEdit(range=52:13-52:13, new_text=': *mut Numbered<T>')], too

### Hierarchy incoming or outgoing calls

In [15]:
item = lsp_types.CallHierarchyItem(
    "Numbered-struct",
    lsp_types.SymbolKind.Function,
    uri=file_uri,
    range=lsp_types.Range(lsp_types.Position(50, 14), lsp_types.Position(50, 24)),
    selection_range=lsp_types.Range(
        lsp_types.Position(50, 14), lsp_types.Position(50, 24)
    ),
)
response = await client.call_hierarchy_incoming_calls_async(
    lsp_types.CallHierarchyIncomingCallsParams(item=item)
)
response

[CallHierarchyIncomingCall(from_=CallHierarchyItem(name='replace', kind=<SymbolKind.Function: 12>, uri='file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs', range=56:4-72:5, selection_range=58:7-58:14, tags=None, detail='fn replace(list: &[Self], value: Self)', data=None), from_ranges=[69:22-69:32])]

In [16]:
response = await client.call_hierarchy_outgoing_calls_async(
    lsp_types.CallHierarchyOutgoingCallsParams(item=item)
)
response

[CallHierarchyOutgoingCall(to=CallHierarchyItem(name='as_ptr', kind=<SymbolKind.Function: 12>, uri='file:///home/lihuan/.rustup/toolchains/nightly-x86_64-unknown-linux-gnu/lib/rustlib/src/rust/library/core/src/slice/mod.rs', range=714:4-747:5, selection_range=745:17-745:23, tags=None, detail='pub const fn as_ptr(&self) -> *const T', data=None), from_ranges=[73:51-74:0]),
 CallHierarchyOutgoingCall(to=CallHierarchyItem(name='add', kind=<SymbolKind.Function: 12>, uri='file:///home/lihuan/.rustup/toolchains/nightly-x86_64-unknown-linux-gnu/lib/rustlib/src/rust/library/core/src/ptr/const_ptr.rs', range=883:4-943:5, selection_range=937:24-937:27, tags=None, detail='pub const unsafe fn add(self, count: usize) -> Self\nwhere\n    T: Sized,', data=None), from_ranges=[54:35-54:38])]

## Diagnostics

### Text Document Diagnostic
`rust-analyzer` does not implement `pull diagnostic` feature which includes `textDocument/diagnostic` and `workspace/diagnostic`, so the only way to get the diagnostic message is through notification from server to client via `textDocument/publishDiagnostic`.

Attention: An open notification must not be sent more than once without a corresponding close notification send before. This means open and close notification must be balanced and the max open count is one.

In [24]:
file_item = lsp_types.TextDocumentItem(
    uri=file_uri, language_id="rust", version=3, text=""
)
# client.text_document_did_open(lsp_types.DidOpenTextDocumentParams(file_item))
# await client.wait_for_notification(lsp_types.TEXT_DOCUMENT_PUBLISH_DIAGNOSTICS)
# client.text_document_did_close(lsp_types.DidCloseTextDocumentParams(file_item))

diagnostics = client.diagnostics[file_uri]
diagnostics

[Diagnostic(range=12:3-12:22, message='function `default_num_threads` is never used\n`#[warn(dead_code)]` on by default', severity=<DiagnosticSeverity.Warning: 2>, code='dead_code', code_description=None, source='rustc', tags=[<DiagnosticTag.Unnecessary: 1>], related_information=None, data={'rendered': 'warning: function `default_num_threads` is never used\n  --> src/main.rs:13:4\n   |\n13 | fn default_num_threads() -> NonZeroUsize {\n   |    ^^^^^^^^^^^^^^^^^^^\n   |\n   = note: `#[warn(dead_code)]` on by default\n\n'}),
 Diagnostic(range=37:4-37:9, message='field `field` is never read\n`Numbered` has a derived impl for the trait `Debug`, but this is intentionally ignored during dead code analysis', severity=<DiagnosticSeverity.Warning: 2>, code='dead_code', code_description=None, source='rustc', tags=[<DiagnosticTag.Unnecessary: 1>], related_information=[DiagnosticRelatedInformation(location=file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15,

### File Diagnostic
Undefined in `rust-analyzer`

In [ ]:
response = await client.text_document_diagnostic_async(
    lsp_types.DocumentDiagnosticParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri)
    )
)
response.items

### Workspace Diagnostic
Undefined in `rust-analyzer`

In [ ]:
response = await client.workspace_diagnostic_async(
    lsp_types.WorkspaceDiagnosticParams(previous_result_ids=[])
)
response.items[0].items

In [25]:
client.log_messages

[]

In [26]:
client.messages

[]